<a href="https://colab.research.google.com/github/JonathonLeiding/fifa-score-analysis/blob/main/fifa_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
class model(nn.Module):
  def __init__(self):
    super(model, self).__init__()

  def forward(self, x):
    return x


In [3]:
url = 'https://raw.githubusercontent.com/JonathonLeiding/fifa-score-analysis/refs/heads/main/jup_notebooks/cleaned_prem_data.csv'
df = pd.read_csv(url)